Can only do this analysis, involving colinearity, for genomes with genome mapped to chromosomes.

Might not be that useful to do it on more species. Antynign not in briggsae clade will be veery distantly related. Fewer orthologs --> less power. Also very likely that rearrangements will occur more than once. Multiple rearrangements of the same type. 

ftp://ftp.wormbase.org/pub/wormbase/releases/WS275/species/c_inopinata/PRJDB5687/

In [1]:
worm='inopinata'
ts_gtf='~/part_ii_project/canonical_genesets/c_inopinata.PRJDB5687.WS275.canonical_geneset.gtf'
worm_fasta_file='c_inopinata.PRJDB5687.WS275.CDS_transcripts.fa'
protein_fasta_file='c_inopinata.PRJDB5687.WS275.protein.fa'

In [1]:
import pandas as pd
import numpy as np
from Bio import SeqIO

In [3]:
chrom_bed_columns=['chrom','chromStart','chromEnd','name',
       'score','strand','thickStart','thickEnd','itemRgb']
bed_12_columns=['chrom','chromStart','chromEnd','name',
       'score','strand','thickStart','thickEnd','itemRgb',
        'blockCount', 'blockSizes', 'blockStart']

# Make transcript to gene map for the worm

Will have to figure this out

In [4]:
with open(worm_fasta_file) as fasta_file:
    descriptions=[]
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):
        descriptions.append(seq_record.description)

In [5]:
transcripts=[]
genes=[]

for i in descriptions:
    transcripts.append(i.split(' gene=')[0])
    genes.append(i.split(' gene=')[1])

In [6]:
ts_gs=pd.DataFrame({'genes':genes, 'transcripts':transcripts})

In [7]:
ts_gs.to_csv('~/part_ii_project/canonical_genesets/'+worm+'_transcripts_and_genes.txt', 
             header=None,index=None, sep='\t')

# Make genes file for the worm

### Make a bed file of transcripts

In [8]:
# convert gtf to genepred to bed

gene_pred_file='~/part_ii_project/canonical_genesets/'+worm+'_genePred'

! ./gtfToGenePred {ts_gtf} {gene_pred_file}

ts_bed='~/part_ii_project/canonical_genesets/'+worm+'_transcripts.bed'

! ./genePredToBed {gene_pred_file} {ts_bed}

In [9]:
ts=pd.read_csv(ts_bed,sep='\t',names=bed_12_columns)

In [10]:
ts

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,itemRgb,blockCount,blockSizes,blockStart
0,Sp34_Chr5,5859,7266,Sp34_50000100.t1,0,-,5859,7266,0,2,"66,183,","0,1224,"
1,Sp34_Chr5,9326,10190,Sp34_50000200.t1,0,+,9326,10190,0,3,"63,56,97,","0,654,767,"
2,Sp34_Chr5,11792,12182,Sp34_50000300.t1,0,+,11792,12182,0,1,"390,","0,"
3,Sp34_Chr5,15692,16990,Sp34_50000400.t1,0,-,15692,16990,0,5,"193,423,236,88,50,","0,378,841,1119,1248,"
4,Sp34_Chr5,21042,23817,Sp34_50000500.t1,0,-,21042,23817,0,5,"16,63,66,117,56,","0,434,1323,2142,2719,"
...,...,...,...,...,...,...,...,...,...,...,...,...
21604,Sp34_ChrX,18172142,18173045,Sp34_X0278900.t1,0,+,18172142,18173045,0,1,"903,","0,"
21605,Sp34_ChrX,18173649,18175323,Sp34_X0279000.t1,0,+,18173649,18175323,0,1,"1674,","0,"
21606,Sp34_ChrX,18176970,18177814,Sp34_X0279100.t1,0,+,18176970,18177814,0,6,"21,59,95,47,82,116,","0,89,215,420,569,728,"
21607,Sp34_ChrX,18178768,18183524,Sp34_X0279200.t1,0,-,18178768,18183524,0,4,"2040,540,270,717,","0,2099,2763,4039,"


### Rerformat chromosome names if necessary

In [11]:
chroms=[i.split('_Chr')[1] for i in list(ts['chrom'])]
ts['chrom']=chroms

arabic_to_rom={'1':'I', '2':'II', '3':'III', '4':'IV', '5':'V', 'X':'X'}
chroms=[arabic_to_rom[i] for i in list(ts['chrom'])]
ts['chrom']=chroms

### Add gene names instead of transcript names and have just 1 row per gene

In [13]:
# get a list of each gene just once

one_of_each_gene=list(set(ts_gs['genes']))

In [14]:
# add the gene IDs to the transcript data frame

total_df=ts.copy()
total_df['gene']=''

for i in range(len(ts_gs)):
    row=total_df.index[total_df['name']==ts_gs['transcripts'][i]]
    total_df.at[row,'gene']=ts_gs['genes'][i]

In [15]:
total_df

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,itemRgb,blockCount,blockSizes,blockStart,gene
0,V,5859,7266,Sp34_50000100.t1,0,-,5859,7266,0,2,"66,183,","0,1224,",Sp34_50000100
1,V,9326,10190,Sp34_50000200.t1,0,+,9326,10190,0,3,"63,56,97,","0,654,767,",Sp34_50000200
2,V,11792,12182,Sp34_50000300.t1,0,+,11792,12182,0,1,"390,","0,",Sp34_50000300
3,V,15692,16990,Sp34_50000400.t1,0,-,15692,16990,0,5,"193,423,236,88,50,","0,378,841,1119,1248,",Sp34_50000400
4,V,21042,23817,Sp34_50000500.t1,0,-,21042,23817,0,5,"16,63,66,117,56,","0,434,1323,2142,2719,",Sp34_50000500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21604,X,18172142,18173045,Sp34_X0278900.t1,0,+,18172142,18173045,0,1,"903,","0,",Sp34_X0278900
21605,X,18173649,18175323,Sp34_X0279000.t1,0,+,18173649,18175323,0,1,"1674,","0,",Sp34_X0279000
21606,X,18176970,18177814,Sp34_X0279100.t1,0,+,18176970,18177814,0,6,"21,59,95,47,82,116,","0,89,215,420,569,728,",Sp34_X0279100
21607,X,18178768,18183524,Sp34_X0279200.t1,0,-,18178768,18183524,0,4,"2040,540,270,717,","0,2099,2763,4039,",Sp34_X0279200


In [16]:
# These are the coding transcripts that we'll be looking at
genes_df=total_df[total_df['gene']!=''].copy()

In [18]:
# Get isoform sizes
genes_df['transcript_size']=list(genes_df['chromEnd']-genes_df['chromStart'])

In [19]:
# make a table of just the longest isoforms of each gene

one_to_one=pd.DataFrame(columns=genes_df.columns)
for i in one_of_each_gene:
    to_compare=genes_df[genes_df['gene']==i]
    longest=to_compare['transcript_size'].argmax()
    row=to_compare.iloc[longest,:]
    to_concat=row.to_frame().T
    one_to_one=pd.concat([one_to_one, to_concat])
    
path='~/part_ii_project/canonical_genesets/'+worm+'_one_to_one_with_transcript_name.bed'
one_to_one.to_csv(path, sep='\t', index=None)

KeyboardInterrupt: 

In [58]:
one_to_one

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,itemRgb,blockCount,blockSizes,blockStart,gene,transcript_size
17709,III,13691959,13693971,Sp34_30238000.t1,0,+,13691959,13693971,0,7,"92,51,112,158,79,150,132,","0,221,323,711,914,1050,1880,",Sp34_30238000,2012
17422,III,12066194,12068990,Sp34_30210020.t1,0,-,12066194,12068990,0,6,"425,124,226,410,22,35,","0,481,650,922,2374,2761,",Sp34_30210020,2796
3217,V,16455650,16458211,Sp34_50307600.t1,0,-,16455650,16458211,0,5,"268,82,92,121,67,","0,328,465,1188,2494,",Sp34_50307600,2561
20864,X,13659551,13664714,Sp34_X0207000.t1,0,-,13659551,13664714,0,10,"167,78,136,84,83,89,117,105,91,106,","0,1900,2025,2207,2695,2837,3042,3819,3979,5057,",Sp34_X0207000,5163
17706,III,13680531,13683192,Sp34_30237700.t1,0,-,13680531,13683192,0,1,"2661,","0,",Sp34_30237700,2661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4831,IV,1684102,1686767,Sp34_40031200.t1,0,+,1684102,1686767,0,7,"25,110,157,368,299,666,145,","0,616,781,987,1405,1805,2520,",Sp34_40031200,2665
5862,IV,8344899,8345855,Sp34_40131210.t1,0,-,8344899,8345855,0,3,"157,196,490,","0,217,466,",Sp34_40131210,956
15672,III,2682125,2682461,Sp34_30039900.t1,0,-,2682125,2682461,0,1,"336,","0,",Sp34_30039900,336
1863,V,9775688,9776590,Sp34_50177500.t1,0,-,9775688,9776590,0,3,"113,94,18,","0,213,884,",Sp34_50177500,902


In [60]:
gene_bed=one_to_one.copy().reset_index(drop=True)
gene_bed['name']=gene_bed['gene']
gene_bed.drop('transcript_size', axis=1).drop('gene', axis=1)

path='~/part_ii_project/canonical_genesets/'+worm+'_one_of_each_gene.bed'
gene_bed.to_csv(path, sep='\t', index=None, header=None)

# Make files of orthologous genes with and without operons

In [16]:
bed_12_columns=['chrom','chromStart','chromEnd','name',
       'score','strand','thickStart','thickEnd','itemRgb',
        'blockCount', 'blockSizes', 'blockStart']

In [17]:
worm_genes=pd.read_csv('~/part_ii_project/canonical_genesets/'+worm+'_one_of_each_gene.bed',
                           names=bed_12_columns, sep='\t')
elegans_genes=pd.read_csv('~/part_ii_project/canonical_genesets/elegans_one_of_each_gene.bed',
                           names=bed_12_columns, sep='\t')

In [18]:
operon_names=['gene_name','operon','blockCount','blockSizes','blockStart',
              'chrom','gene_end','gene_start','itemRgb','score','strand','thickEnd',
              'thickStart','position_in_operon']

operons=pd.read_csv('~/part_ii_project/operon_information/every_gene_and_its_operon',names=operon_names,sep='\t')

indices=[]
for index, row in elegans_genes.iterrows():
    if row['name'] not in list(operons['gene_name']):indices.append(index)
        
elegans_genes_no_ops=elegans_genes.iloc[indices,:].copy().reset_index(drop=True)

In [19]:
len(elegans_genes_no_ops), len(elegans_genes)

(16430, 20040)

In [69]:
orthologous_genes=pd.read_csv('~/part_ii_project/orthologs/elegans_'+worm+'_with_ortholog_type.txt', sep='\t')

one2one=orthologous_genes[orthologous_genes['Homology type']=='ortholog_one2one'].copy()
orth_df=one2one.drop('Homology type', axis=1).drop('Genome project', axis=1)
orth_df.columns=['elegans',worm]

orth_df

,elegans,inopinata
0,WBGene00000001,Sp34_10257700
1,WBGene00000002,Sp34_40340700
2,WBGene00000003,Sp34_50242000
3,WBGene00000004,Sp34_X0099800
5,WBGene00000006,Sp34_10138500
...,...,...
48841,WBGene00305086,Sp34_20266820
48859,WBGene00305157,Sp34_20128400
48875,WBGene00305999,Sp34_30169600
48877,WBGene00306001,Sp34_30207200


In [70]:
elegans_to_worm={}

for index, row in orth_df.iterrows():
    elegans_to_worm[row['elegans']]=row[worm]

In [22]:
# WITH OPERONS
# this goes through all the genes in the c. elegans gene list and takes only those genes where:
# 1) they are present in the list of 1:1 orthologs
# 2) their briggsae orthologue is present in the briggsae gene list (some discrepances here due to release #)

# it also gathers the indices necessary to reorder the briggsae df so that each ortholog in elegans/briggsae is at the same index

elegans_good_genes=[]
elegans_idx=[]
worm_good_genes=[]
worm_idx=[]
for index, row in elegans_genes.iterrows():
    take=orth_df[orth_df['elegans']==row['name']].copy().reset_index(drop=True)
    if len(take)>0:
        worm_take=worm_genes[worm_genes['name']==take[worm][0]].copy().reset_index(drop=False)
        if len(worm_take)>0:
            elegans_good_genes.append(row['name'])
            elegans_idx.append(index)
            worm_good_genes.append(worm_take['name'][0])
            worm_idx.append(worm_take['index'][0])
            
elegans_final_with_ops=elegans_genes.iloc[elegans_idx,:].copy().reset_index(drop=True)
worm_final_with_ops=worm_genes.iloc[worm_idx,:].copy().reset_index(drop=True)

In [23]:
# NO OPERONS
# this goes through all the genes in the c. elegans gene list and takes only those genes where:
# 1) they are present in the list of 1:1 orthologs
# 2) their briggsae orthologue is present in the briggsae gene list (some discrepances here due to release #)

# it also gathers the indices necessary to reorder the briggsae df so that each ortholog in elegans/briggsae is at the same index

elegans_good_genes=[]
elegans_idx=[]
worm_good_genes=[]
worm_idx=[]
for index, row in elegans_genes_no_ops.iterrows():
    take=orth_df[orth_df['elegans']==row['name']].copy().reset_index(drop=True)
    if len(take)>0:
        worm_take=worm_genes[worm_genes['name']==take[worm][0]].copy().reset_index(drop=False)
        if len(worm_take)>0:
            elegans_good_genes.append(row['name'])
            elegans_idx.append(index)
            worm_good_genes.append(worm_take['name'][0])
            worm_idx.append(worm_take['index'][0])
            
elegans_final_no_ops=elegans_genes_no_ops.iloc[elegans_idx,:].copy().reset_index(drop=True)
worm_final_no_ops=worm_genes.iloc[worm_idx,:].copy().reset_index(drop=True)

In [24]:
worm_final_with_ops.to_csv('~/part_ii_project/canonical_genesets/'+worm+'_to_compare.bed', 
                     header=None, index=None, sep='\t')
elegans_final_with_ops.to_csv('~/part_ii_project/canonical_genesets/elegans_for_'+worm+'_to_compare.bed',
                             header=None, index=None, sep='\t')

worm_final_no_ops.to_csv('~/part_ii_project/canonical_genesets/'+worm+'_orths_no_ops.bed', 
                     header=None, index=None, sep='\t')
elegans_final_no_ops.to_csv('~/part_ii_project/canonical_genesets/elegans_for_'+worm+'_orths_no_ops.bed',
                           header=None, index=None, sep='\t')

In [25]:
elegans_final_no_ops

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,itemRgb,blockCount,blockSizes,blockStart
0,I,4114,10606,WBGene00022277,0,-,4220,10148,0,6,"244,102,291,120,136,77,","0,1080,1922,5612,5980,6415,"
1,I,11494,16837,WBGene00022276,0,+,11640,16585,0,4,"67,72,210,365,","0,123,3456,4978,"
2,I,17482,26781,WBGene00022278,0,-,17910,26778,0,12,"476,110,227,208,117,127,195,162,200,154,92,49,","0,523,1532,2788,3365,3530,7168,7446,7790,8888,..."
3,I,27588,33150,WBGene00022279,0,-,28279,32482,0,6,"817,268,656,304,131,114,","0,1511,2180,4180,4823,5448,"
4,I,93021,94884,WBGene00021676,0,+,93091,94714,0,6,"115,375,407,145,300,263,","0,160,587,1042,1247,1600,"
...,...,...,...,...,...,...,...,...,...,...,...,...
8107,X,17662444,17665357,WBGene00003744,0,+,17662462,17664159,0,4,"116,146,89,430,","0,876,1629,2483,"
8108,X,17671021,17680793,WBGene00020732,0,+,17679538,17680396,0,14,"138,86,90,150,469,304,258,459,222,286,72,418,1...","0,184,2369,3096,3337,3924,4343,4742,5249,5751,..."
8109,X,17680820,17684811,WBGene00044772,0,-,17680910,17682294,0,3,"205,95,56,","0,1379,3935,"
8110,X,17695231,17701744,WBGene00017625,0,+,17696474,17701424,0,7,"88,250,236,221,324,132,491,","0,1241,2747,3028,3474,4465,6022,"


In [26]:
worm_final_no_ops

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,itemRgb,blockCount,blockSizes,blockStart
0,I,12509907,12514217,Sp34_10225700,0,-,12509907,12514217,0,4,"141,102,291,162,","0,295,445,4148,"
1,I,12514398,12517578,Sp34_10225800,0,+,12514398,12517578,0,5,"5,6,62,207,113,","0,1319,1377,2034,3067,"
2,I,5345749,5348704,Sp34_10093510,0,-,5345749,5348704,0,12,"78,110,227,208,144,127,195,162,194,139,113,46,","0,127,316,603,888,1241,1473,1725,1936,2306,274..."
3,I,12519808,12522472,Sp34_10225900,0,-,12519808,12522472,0,4,"126,256,966,71,","0,181,1228,2593,"
4,II,18824098,18826602,Sp34_20323200,0,-,18824098,18826602,0,5,"105,439,407,708,72,","0,154,1171,1631,2432,"
...,...,...,...,...,...,...,...,...,...,...,...,...
8107,X,2218882,2219284,Sp34_X0029400,0,-,2218882,2219284,0,3,"17,140,98,","0,119,304,"
8108,X,2205166,2213498,Sp34_X0029300,0,-,2205166,2213498,0,15,"112,111,68,57,102,75,286,222,525,243,304,502,9...","0,187,950,1142,1395,2484,2686,3154,3428,4115,4..."
8109,X,2203666,2204596,Sp34_X0029200,0,+,2203666,2204596,0,3,"36,95,115,","0,531,815,"
8110,X,187336,194193,Sp34_X0002300,0,-,187336,194193,0,6,"234,132,324,221,236,242,","0,321,2777,4986,5730,6615,"


In [27]:
elegans_to_worm['WBGene00022278']

'Sp34_10093510'

# Get protein sequence per gene

'~/part_ii_project/protein_sets/get_protein_sequence_per_gene'

In [99]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [71]:
path='~/part_ii_project/canonical_genesets/'+worm+'_one_to_one_with_transcript_name.bed'
one_to_one=pd.read_csv(path, sep='\t')

In [73]:
# may have to remake the 'name column' so that it is in the same format as transcript names in the protein fasta

In [97]:
# go through the fasta file downloaded from the ftp site
# take only those records with an id that is present in the one_to_one df column with names of longest transcripts

to_take=[]

path=protein_fasta_file

to_take=[record for record in SeqIO.parse(path, 'fasta') if record.id in list(one_to_one['name'])]

In [75]:
len(to_take)

21443

In [76]:
transcript_id_path='/home/helena_hatrick/part_ii_project/protein_sets/'+worm+'_longest_protein_sequences_transcript_id.fasta'
SeqIO.write(to_take, transcript_id_path, "fasta")

In [81]:
t_to_g={}
for index, row in one_to_one.iterrows():
    t_to_g[row['name']]=row['gene']

In [83]:
# Replace transcript ids with gene ids

gene_ids=[record for record in SeqIO.parse(transcript_id_path,'fasta')]

for record in gene_ids:
    record.id=t_to_g[record.id]

In [86]:
gene_id_path='/home/helena_hatrick/part_ii_project/protein_sets/'+worm+'_longest_protein_sequences_gene_id.fasta'
SeqIO.write(gene_ids,gene_id_path,"fasta")

21443

In [89]:
path='/home/helena_hatrick/part_ii_project/protein_sets/'+worm+'_longest_protein_sequences_gene_id.fasta'
    
ids=[record.id for record in SeqIO.parse(path,'fasta')]

seqs=[record.seq for record in SeqIO.parse(path,'fasta')]

initial_descriptions=[record.description for record in SeqIO.parse(path,'fasta')]
descriptions=[]
for description in initial_descriptions:
    split=description.split(' ')[2:]
    joined=' '.join([str(x) for x in split])
    descriptions.append(joined)

In [106]:
recs=[]
for i in range(len(ids)):
    
    rec=SeqRecord(seqs[i],id=ids[i], description=descriptions[i])
    recs.append(rec)

path='/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/'+worm+'_gene_id_longest_protein_sequence_with_description.fasta'
SeqIO.write(recs,path,'fasta')

21443

In [112]:
recs=[]
for i in range(len(ids)):
    
    rec=SeqRecord(seqs[i],id=ids[i])
    recs.append(rec)

path='/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/'+worm+'_gene_id_longest_protein_sequence_no_description.fasta'
SeqIO.write(recs,path,'fasta')

21443

# Run BLASTp

'~/part_ii_project/run_blastp'

In [24]:
target_worm='elegans'
query_worm=worm

query_fasta='/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/'+query_worm+'_gene_id_longest_protein_sequence_no_description.fasta'
target_fasta='/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/'+target_worm+'_gene_id_longest_protein_sequence_no_description.fasta'
target_db='/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/'+target_worm+'_blastdb'

! makeblastdb -in {target_fasta} -out {target_db} -dbtype prot

out_file='/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/'+target_worm+'_target_vs_'+query_worm+'_query.txt'

! blastp -db {target_db} -query {query_fasta} -out {out_file} -outfmt 6

In [82]:
target_worm=worm
query_worm='elegans'

query_fasta='/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/'+query_worm+'_gene_id_longest_protein_sequence_no_description.fasta'
target_fasta='/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/'+target_worm+'_gene_id_longest_protein_sequence_no_description.fasta'
target_db='/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/'+target_worm+'_blastdb'

! makeblastdb -in {target_fasta} -out {target_db} -dbtype prot

out_file='/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/'+target_worm+'_target_vs_'+query_worm+'_query.txt'

! blastp -db {target_db} -query {query_fasta} -out {out_file} -outfmt 6



Building a new DB, current time: 03/03/2021 23:37:35
New DB name:   /home/helena_hatrick/part_ii_project/protein_sets/run_blastp/inopinata_blastdb
New DB title:  /home/helena_hatrick/part_ii_project/protein_sets/run_blastp/inopinata_gene_id_longest_protein_sequence_no_description.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /home/helena_hatrick/part_ii_project/protein_sets/run_blastp/inopinata_blastdb
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 21443 sequences in 4.69531 seconds.


# Make file of RBH ortholog pairs and their e values from BLASTp

In [83]:
# read in the blast outputs

names=['qseqid','sseqid','pident','length','mismatch','gapopen','qstart','qend','sstart','send','evalue','bitscore']
path_stem='/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/'

elegans_vs_worm=pd.read_csv(path_stem+'elegans_target_vs_'+worm+'_query.txt',sep='\t',names=names)

worm_vs_elegans=pd.read_csv(path_stem+worm+'_target_vs_elegans_query.txt',sep='\t',names=names)

In [84]:
operon_options=['with', 'no']
which_target=['elegans', worm]

In [85]:
best_hits={}

for target in which_target:

    if target=='elegans': df=elegans_vs_worm
    if target==worm: df=worm_vs_elegans
    top_hit_ixs=[]

    for num, query_gene in enumerate(list(set(df['qseqid']))):
        if num%2000==0:print(num)
        subdf=df[df['qseqid']==query_gene].copy()
        top_hit_ixs.append(list(subdf.index)[0])

    best_hits[target]=df.iloc[top_hit_ixs, :].copy().reset_index(drop=True)

0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
0
2000
4000
6000
8000
10000
12000
14000
16000
18000


I will be using the dataset with elegans as target for all future things, not that it should matter.

In [86]:
best_hits['elegans']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,Sp34_50429700,WBGene00044183,39.474,38,16,1,81,111,2,39,2.000000e+00,25.8
1,Sp34_30105200,WBGene00011734,97.183,142,4,0,1,142,1,142,6.170000e-101,284.0
2,Sp34_20183600,WBGene00010711,27.389,157,95,8,137,290,1,141,5.810000e-07,48.1
3,Sp34_10273500,WBGene00007700,74.474,380,82,4,53,432,18,382,0.000000e+00,577.0
4,Sp34_50230810,WBGene00019203,87.662,154,19,0,1,154,1,154,1.190000e-102,290.0
...,...,...,...,...,...,...,...,...,...,...,...,...
21417,Sp34_50402510,WBGene00012615,75.556,90,22,0,1,90,1,90,2.270000e-41,130.0
21418,Sp34_20217500,WBGene00004038,91.790,1352,105,4,1,1348,1,1350,0.000000e+00,2574.0
21419,Sp34_X0160500,WBGene00001425,85.430,151,22,0,1,151,1,151,2.520000e-82,238.0
21420,Sp34_20091000,WBGene00011464,23.784,185,121,7,27,197,47,225,6.000000e-03,37.0


In [91]:
rbh_indices=[]

e_target=best_hits['elegans']
x_target=best_hits[worm]

for index, row in e_target.iterrows():

    hit = row['sseqid']
    test=x_target[x_target['qseqid']==hit].reset_index(drop=True)
    if test['sseqid'][0]==row['qseqid']:rbh_indices.append(index)

rbhs=e_target.iloc[rbh_indices, :].copy().reset_index(drop=True)

In [92]:
rbhs

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,Sp34_50429700,WBGene00044183,39.474,38,16,1,81,111,2,39,2.000000e+00,25.8
1,Sp34_30105200,WBGene00011734,97.183,142,4,0,1,142,1,142,6.170000e-101,284.0
2,Sp34_10273500,WBGene00007700,74.474,380,82,4,53,432,18,382,0.000000e+00,577.0
3,Sp34_50230810,WBGene00019203,87.662,154,19,0,1,154,1,154,1.190000e-102,290.0
4,Sp34_20044610,WBGene00022448,90.062,161,16,0,1,161,1,161,1.400000e-110,310.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12639,Sp34_50277900,WBGene00011284,86.014,143,20,0,1,143,1,143,2.470000e-91,260.0
12640,Sp34_50402510,WBGene00012615,75.556,90,22,0,1,90,1,90,2.270000e-41,130.0
12641,Sp34_20217500,WBGene00004038,91.790,1352,105,4,1,1348,1,1350,0.000000e+00,2574.0
12642,Sp34_X0160500,WBGene00001425,85.430,151,22,0,1,151,1,151,2.520000e-82,238.0


In [93]:
worm_final_with_ops=pd.read_csv('~/part_ii_project/canonical_genesets/'+worm+'_to_compare.bed', 
                     names=bed_12_columns, sep='\t')
elegans_final_with_ops=pd.read_csv('~/part_ii_project/canonical_genesets/elegans_for_'+worm+'_to_compare.bed',
                             names=bed_12_columns, sep='\t')

worm_final_no_ops=pd.read_csv('~/part_ii_project/canonical_genesets/'+worm+'_orths_no_ops.bed', 
                     names=bed_12_columns, sep='\t')
elegans_final_no_ops=pd.read_csv('~/part_ii_project/canonical_genesets/elegans_for_'+worm+'_orths_no_ops.bed',
                           names=bed_12_columns, sep='\t')

In [94]:
ortholog_rbhs={}

for is_ops in operon_options:
    
    df=rbhs.copy()
    ortholog_rbh_indices=[]
    if is_ops=='with': ortholog_df=elegans_final_with_ops
    if is_ops=='no':ortholog_df=elegans_final_no_ops
    
    for index, row in df.iterrows():
        if row['sseqid'] in set(ortholog_df['name']):
            if elegans_to_worm[row['sseqid']]==row['qseqid']: ortholog_rbh_indices.append(index)
            
    ortholog_rbhs[is_ops]=df.iloc[ortholog_rbh_indices,:].copy().reset_index(drop=True)

In [99]:
ortholog_rbhs['no']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,Sp34_50230810,WBGene00019203,87.662,154,19,0,1,154,1,154,1.190000e-102,290.0
1,Sp34_20153100,WBGene00008052,74.010,404,105,0,45,448,1,404,0.000000e+00,639.0
2,Sp34_50092700,WBGene00021493,83.491,424,69,1,11,433,10,433,0.000000e+00,751.0
3,Sp34_X0144500,WBGene00008094,86.604,739,98,1,1,739,1,738,0.000000e+00,1335.0
4,Sp34_20032600,WBGene00012602,83.824,68,11,0,1,68,1,68,6.790000e-41,127.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7910,Sp34_X0248300,WBGene00006475,93.262,564,38,0,43,606,1,564,0.000000e+00,1073.0
7911,Sp34_50402510,WBGene00012615,75.556,90,22,0,1,90,1,90,2.270000e-41,130.0
7912,Sp34_20217500,WBGene00004038,91.790,1352,105,4,1,1348,1,1350,0.000000e+00,2574.0
7913,Sp34_X0160500,WBGene00001425,85.430,151,22,0,1,151,1,151,2.520000e-82,238.0


In [102]:
ortholog_rbhs_neat={}

for is_ops in operon_options:
    
    df=ortholog_rbhs[is_ops]
    neat_df=pd.DataFrame({'query_gene':df['qseqid'], 'target_gene':df['sseqid'], 'e_value':df['evalue']})
    path='~/part_ii_project/orthologs/'+worm+'_query_121_ortholog_rbhs_'+is_ops+'_ops'
    neat_df.to_csv(path, header=['query','target','e_value'], index=None)
    ortholog_rbhs_neat[is_ops]=neat_df

In [103]:
ortholog_rbhs_neat['with']

,query_gene,target_gene,e_value
0,Sp34_30105200,WBGene00011734,6.170000e-101
1,Sp34_50230810,WBGene00019203,1.190000e-102
2,Sp34_20044610,WBGene00022448,1.400000e-110
3,Sp34_20153100,WBGene00008052,0.000000e+00
4,Sp34_50092700,WBGene00021493,0.000000e+00
...,...,...,...
10755,Sp34_50277900,WBGene00011284,2.470000e-91
10756,Sp34_50402510,WBGene00012615,2.270000e-41
10757,Sp34_20217500,WBGene00004038,0.000000e+00
10758,Sp34_X0160500,WBGene00001425,2.520000e-82


In [5]:
worm='inopinata'
is_ops='with'
test=pd.read_csv('~/part_ii_project/orthologs/'+worm+'_query_121_ortholog_rbhs_'+is_ops+'_ops')
test

,query,target,e_value
0,Sp34_30105200,WBGene00011734,6.170000e-101
1,Sp34_50230810,WBGene00019203,1.190000e-102
2,Sp34_20044610,WBGene00022448,1.400000e-110
3,Sp34_20153100,WBGene00008052,0.000000e+00
4,Sp34_50092700,WBGene00021493,0.000000e+00
...,...,...,...
10755,Sp34_50277900,WBGene00011284,2.470000e-91
10756,Sp34_50402510,WBGene00012615,2.270000e-41
10757,Sp34_20217500,WBGene00004038,0.000000e+00
10758,Sp34_X0160500,WBGene00001425,2.520000e-82


In [8]:
worm='briggsae'
is_ops='with'
test=pd.read_csv('~/part_ii_project/orthologs/'+worm+'_query_121_ortholog_rbhs_'+is_ops+'_ops')
test

FileNotFoundError: [Errno 2] No such file or directory: '/home/helena_hatrick/part_ii_project/orthologs/briggsae_query_121_ortholog_rbhs_with_ops'

'~/part_ii_project/get_evalues_no_ops'

'~/part_ii_project/get_evalues_with_ops'